Como informado em sala de aula, a nota da unidade 1 deverá ser enviada indivudalmente aqui e apresentada no dia 08/05/2025.

A atividade 1 consistirá no envio do link de uma pasta compartilhada no google drive ou um repositório no github contendo:

1 - Datasets e todos os artefatos utilizados

2 - Cadernos jupyter com o processo aplicado em uma pipeline de supervisão fraca. 

O caderno deverá primar pela qualidade, incluindo:

a) Análise exploratória de dados

b) Storytelling

c) Explicação detalhada de cada etapa

 

As células devem estar EXECUTADAS, com a saída registrada no caderno.

 

 A atividade será avaliada pela qualidade técnica e esmero na construção do caderno.

 

Um abraço,

 

Elias 

Obs: Certifique-se de que a pasta/repositório do Github esteja acessível para qualquer pessoa com um link.

In [1]:
import pandas as pd

df_val_test = pd.read_csv('dataset/labeledTokenClassification.csv')
df_unlabeled = pd.read_csv('dataset/unlabeledMadeiraDescricao.csv')

In [2]:
df_val_test = df_val_test[df_val_test.status == 'completed']

In [3]:
from sklearn.model_selection import train_test_split

df_val, df_test = train_test_split(
    df_val_test,
    test_size=0.5,       
    random_state=1, # Necessário a mudança caso utilizar o conjunto de teste
    shuffle=True         
)
print(len(df_val), len(df_test))

282 282


In [4]:
len(df_unlabeled)

21010

## Primeiro Passo 

In [5]:
import skweak
import spacy

nlp = spacy.load("pt_core_news_lg")

In [6]:
spacy_docs_train = list(nlp.pipe(df_unlabeled["text"].values))

spacy_docs_dev = list(nlp.pipe(df_val["text"].values))

spacy_docs_test = list(nlp.pipe(df_test["text"].values))

skweak.utils.docbin_writer(spacy_docs_train, "artefatos/spacy_docs_train.bin")
skweak.utils.docbin_writer(spacy_docs_dev, "artefatos/spacy_docs_dev.bin")
skweak.utils.docbin_writer(spacy_docs_test, "artefatos/spacy_docs_test.bin")

Write to artefatos/spacy_docs_train.bin...done
Write to artefatos/spacy_docs_dev.bin...done
Write to artefatos/spacy_docs_test.bin...done


In [7]:
# Load the training documents
spacy_docs_train = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_train.bin", spacy_model_name="pt_core_news_lg"
)

# Load the validation documents
spacy_docs_dev = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_dev.bin", spacy_model_name="pt_core_news_lg"
)

# Load the test documents
spacy_docs_test = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_dev.bin", spacy_model_name="pt_core_news_lg"
)

# Convert the loaded documents to lists
# This step ensures that the documents are in a list format, which is easier to work with in subsequent steps
spacy_docs_train = list(spacy_docs_train)
spacy_docs_dev = list(spacy_docs_dev)
spacy_docs_test = list(spacy_docs_test)

In [8]:
spacy_docs_train[:2]

[angelim verm aproveitamento ripas, caibro mixto 4,5mt]

## Passo 2

Adicionar o nom cientifico das especies


In [9]:
madeiras_dicionario = {
    'acacia': ['angico'],
    'acapu': ['angelim de folha larga'],
    'amapa': [],
    'amescla': ['amesclao', 'breu', 'morcegueira', 'sucuruba'],
    'andiroba': ['angirova', 'carapa'],
    'angelim amargoso': ['angelim amarelo', 'fava', 'fava amargosa', 'faveira'],
    'angelim pedra': ['angelim verm', 'angelim vermelho', 'angelim falso', 'faveira ferro'],
    'angelim indefinido': ['angelim', 'angelin', 'ang'],
    'cedrinho': ['quaruba', 'quaruvatinga', 'mandioqueira', 'quaruba vermelha'],
    'cedro': [],
    'cerejeira': [],
    'cetim': ['pau amarelo', 'pau cetim', 'piquia cetim', 'garapa', ' muirajuba'],
    'coco pau': ['baru'],
    'copaiba': ['cupauba', 'copauva'],
    'cumaru': ['cumaru de folha grande', 'cumbar', 'cumbaru roxo'],
    'cupiuba': ['copiuba', 'copiuva'],
    'eucalipto': ['euc'],
    'freijo': [],
    'garapeira': [],
    'goiabao': ['abiu', 'abiurana'],
    'guajara': ['currupixa', 'angico-bravo', 'angico-cedro', 'angico-rosa', 'curupia'],
    'ipe': [],
    'itauba': [],
    'jatoba': ['jatai', 'utai', 'jati'],
    'jequitiba': ['cachimbeiro', 'estopeira', 'coatinga', 'bingueiro'],
    'louro amarelo': ['louro pardo, louro abacate', 'louro inhamui'],
    'louro canela': ['louro rosa', 'louro gamela', 'louro vermelho'],
    'louro indefinido': ['louro'],
    'louro preto': [],
    'marupa': [],
    'massaranduba': ['macaranduba', 'masaranduba', 'maparajuba', 'aparaui', 'paraju', 'mass'],
    'matamata': [],
    'melanceira': ['sucupira-pepino'],
    'mogno': [],
    'muiracatiara': [],
    'para para': ['morototo', 'mandioqueira'],
    'pau louro': [],
    'pau mulato': [],
    'peroba': ['amapa amargoso', 'parahancornia'],
    'pinho': ['araucaria'],
    'pinus': ['pinheiro', 'pinnus'],
    'piquia': ['piqui', 'piquiarana', 'pequia'],
    'roxinho': ['pau roxo', 'guarabu', 'pau ferro'],
    'sapucaia': ['jarana'],
    'sucupira': ['cutiba', 'sapupira', 'sebepira'],
    'sumauma': ['ceiba'],
    'tanimbuca': ['amarelao', 'mirindiba', 'embiridiba', 'cuiarana ,carana', 'mangue'],
    'tatajuba': ['amarelo', 'amarelinho', 'garrote', 'bagaceira'],
    'tauari': ['imbirema', 'estopeiro', 'toari'],
    'taxi': [],
    'timborana': ['timbauba', 'angico'],
    'uxi': ['axua', 'cumate', 'paruru'],
    'mista': ['misto', 'mixta', 'mix'],
    'virola': ['ucuuba']
}

In [78]:
cientifico_madeiras_dicionario = {
    'acapu': ['vouacapoua americana', 'vouacapoua'],
    'amapa': ['parahancornia amapa', 'parahancornia'],
    'amescla': ['protium heptaphyllum', 'protium', 'heptaphyllum'],
    'andiroba': ['carapa guianensis', 'carapa', 'guianensis'],
    'angelim amargoso': ['vatairea sericea', 'vatairea', 'sericea'],
    'angelim pedra': ['hymenolobium petraeum', 'hymenolobium', 'petraeum'],
    'angelim indefinido': ['dinizia excelsa', 'dinizia', 'excelsa'],
    'cedrinho': ['erisma uncinatum', 'erisma', 'uncinatum'],
    'cedro': ['cedrela fissilis', 'cedrela', 'fissilis'],
    'cerejeira': ['amburana cearensis', 'amburana', 'cearensis'],
    'cetim': ['cedrelinga cateniformis', 'cedrelinga', 'cateniformis'],
    'coco pau': ['orbygnia phalerata', 'orbygnia', 'phalerata'],
    'copaiba': ['copaifera langsdorffii', 'copaifera', 'langsdorffii'],
    'cumaru': ['dipteryx odorata', 'dipteryx', 'odorata'],
    'cupiuba': ['goupia glabra', 'goupia', 'glabra'],
    'eucalipto': ['eucalyptus'],
    'freijo': ['cordia goeldiana', 'cordia', 'goeldiana'],
    'garapeira': ['apuleia leiocarpa', 'apuleia', 'leiocarpa'],
    'goiabao': ['qualea'],
    'guajara': ['terminalia amazonia', 'terminalia', 'amazonia'],
    'ipe': ['handroanthus'],
    'itauba': ['mezilaurus itauba', 'mezilaurus', 'itauba'],
    'jatoba': ['hymenaea courbaril', 'hymenaea', 'courbaril'],
    'jequitiba': ['cariniana legalis', 'cariniana', 'legalis'],
    'louro amarelo': ['ocotea'],
    'louro canela': ['nectandra'],
    'louro indefinido': ['ocotea'],
    'louro preto': ['ocotea porosa', 'ocotea', 'porosa'],
    'marupa': ['simarouba amara', 'simarouba', 'amara'],
    'massaranduba': ['manilkara huberi', 'manilkara', 'huberi'],
    'matamata': ['eschweilera'],
    'melanceira': ['trattinnickia burserifolia', 'trattinnickia', 'burserifolia'],
    'mogno': ['swietenia macrophylla', 'swietenia', 'macrophylla'],
    'muiracatiara': ['astronium lecointei', 'astronium', 'lecointei'],
    'para para': ['jacaranda copaia', 'jacaranda', 'copaia'],
    'pau louro': ['ocotea'],
    'pau mulato': ['calycophyllum spruceanum', 'calycophyllum', 'spruceanum'],
    'peroba': ['aspidosperma'],
    'pinho': ['araucaria angustifolia', 'araucaria', 'angustifolia'],
    'pinus': ['pinus'],
    'piquia': ['caryocar villosum', 'caryocar', 'villosum'],
    'roxinho': ['peltogyne'],
    'sapucaia': ['lecythis pisonis', 'lecythis', 'pisonis'],
    'sucupira': ['bowdichia virgilioides', 'bowdichia', 'virgilioides'],
    'sumauma': ['ceiba pentandra', 'ceiba', 'pentandra'],
    'tanimbuca': ['buchenavia tetraphylla', 'buchenavia', 'tetraphylla'],
    'tatajuba': ['bagassa guianensis', 'bagassa', 'guianensis'],
    'tauari': ['couratari'],
    'taxi': ['sclerolobium'],
    'timborana': ['enterolobium schomburgkii', 'enterolobium', 'schomburgkii'],
    'uxi': ['endopleura uchi', 'endopleura', 'uchi'],
    'virola': ['virola']
}

In [79]:
apresentacoes_dicionario ={
                            'alizar': [],
                            'barrote':[],
                            'caibro':['caibrinho'],
                            'frechal': [],
                            'janela': ['janelao'],
                            'linha':[],
                            'mourao': [],
                            'quadrado': ['bloco', 'file', 'quadradinho'],
                            'piquete': [],
                            'pontalete': [],
                            'porta': ['portal', 'aduela'],
                            'prancha':['pranchoes', 'pranchinha'],
                            'ripa':['ripao'],
                            'tabua':[],
                            'taipa':[],
                            'sarrafo':['serrafo'],
                            'viga':['virga','vigota', 'poste']
                            }

In [80]:
def adc_plural(apresentacoes_dict):
    updated_dict = {}
    for key, synonyms in apresentacoes_dict.items():
        pluralized_synonyms = []
        
        if key[-1].lower() in{'a', 'e', 'i', 'o', 'u'}:
            plural = key + "s"
        
            if plural not in synonyms:
                pluralized_synonyms.append(plural)

        for term in synonyms:
            pluralized_synonyms.append(term)
            
            if term and term[-1].lower() in {'a', 'e', 'i', 'o', 'u'}:
                if " " in term:
                    parts = term.split()
                    parts[-1] += "s"
                    plural = " ".join(parts)
                else:
                    plural = term + "s"
                pluralized_synonyms.append(plural)
        
        seen = set()
        unique_synonyms = [t for t in pluralized_synonyms 
                          if not (t in seen or seen.add(t))]
        
        updated_dict[key] = unique_synonyms
        
    return updated_dict

apresentacoes_dicionario = adc_plural(apresentacoes_dicionario)

In [136]:
import re
## (PALAVRA) pode conter uma palavra | PALAVRA precisa conter uma palavra.
padroes = [
    # NUM X NUM X NUM (PALAVRA) -- 2.5 x 3 x 4 cm
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]*)', 
    # NUM PALAVRA X NUM PALAVRA X NUM PALAVRA -- 2.5 cm x 3 m x 4 mm
    r'\b(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\b', 
    # NUM X NUM (PALAVRA) -- 2 x 3 cm
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]*)', 
    # NUM PALAVRA X NUM PALAVRA -- 2cm x 3m
    r'\b(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\b',
    # NUM X NUM -- 2.5 x 3
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\b',
    # NUM (m, cm, mm, metro, pol)
    r'\b(\d+[\.,]?\d*)\s*(m|cm|mm|metro|metros|pol)\b'
]

padroes_compilados = [re.compile(padrao, re.IGNORECASE) for padrao in padroes]

In [137]:
def detectar_dimensao(doc):
    text = doc.text
    for padrao in padroes_compilados:
        for match in padrao.finditer(text):
            start_char, end_char = match.span()
            span = doc.char_span(start_char, end_char, alignment_mode="expand")
            if span:
                yield span.start, span.end, "DIMENSAO"

In [138]:
from skweak import heuristics
dimensao_annotator = heuristics.FunctionAnnotator(
    "dimensao_annotator", 
    detectar_dimensao,
)

In [139]:
from skweak.gazetteers import GazetteerAnnotator, Trie
from skweak.base import CombinedAnnotator

In [140]:
especie_terms = []
for key, synonyms in madeiras_dicionario.items():
    especie_terms.append(key)
    especie_terms.extend(synonyms)

cientifico_especie_terms = []
for key, cientifico in cientifico_madeiras_dicionario.items():
    cientifico_especie_terms.extend(cientifico)

corte_terms = []
for key, synonyms in apresentacoes_dicionario.items():
    corte_terms.append(key)
    corte_terms.extend(synonyms)

In [141]:
def build_trie(term_list):
    """Convert a list of terms into a trie of token sequences"""
    trie = Trie()
    for term in term_list:
        tokens = term.split()
        trie.add(tokens)
    return trie

especie_trie = build_trie(especie_terms)
cientifico_especie_trie = build_trie(cientifico_especie_terms)
corte_trie = build_trie(corte_terms)

In [142]:
cientifico_especie_trie

[['vouacapoua'], ['vouacapoua', 'americana'], ['parahancornia'], ['parahancornia', 'amapa'], ['protium'], ['protium', 'heptaphyllum'], ['heptaphyllum'], ['carapa'], ['carapa', 'guianensis'], ['guianensis'], ['vatairea'], ['vatairea', 'sericea'], ['sericea'], ['hymenolobium'], ['hymenolobium', 'petraeum'], ['petraeum'], ['dinizia'], ['dinizia', 'excelsa'], ['excelsa'], ['erisma'], ['erisma', 'uncinatum'], ['uncinatum'], ['cedrela'], ['cedrela', 'fissilis'], ['fissilis'], ['amburana'], ['amburana', 'cearensis'], ['cearensis'], ['cedrelinga'], ['cedrelinga', 'cateniformis'], ['cateniformis'], ['orbygnia'], ['orbygnia', 'phalerata'], ['phalerata'], ['copaifera'], ['copaifera', 'langsdorffii'], ['langsdorffii'], ['dipteryx'], ['dipteryx', 'odorata'], ['odorata'], ['goupia'], ['goupia', 'glabra'], ['glabra'], ['eucalyptus'], ['cordia'], ['cordia', 'goeldiana'], ['goeldiana'], ['apuleia'], ['apuleia', 'leiocarpa'], ['leiocarpa'], ['qualea'], ['terminalia'], ['terminalia', 'amazonia'], ['amazo

In [143]:
especie_annotator = GazetteerAnnotator("especie_annotator", {"ESPECIE": especie_trie})
cientifico_especie_annotator = GazetteerAnnotator("cientifico_especie_annotator", {"ESPECIE": cientifico_especie_trie})
corte_annotator = GazetteerAnnotator("corte_annotator", {"CORTE": corte_trie})

In [144]:
combined = CombinedAnnotator()
combined.add_annotator(especie_annotator)
combined.add_annotator(cientifico_especie_annotator)
combined.add_annotator(corte_annotator)
combined.add_annotator(dimensao_annotator)

In [145]:
doc = nlp("madeira serrada massaranduba (manilkara huberi) 2.5 metro 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm")
combined(doc)


madeira serrada massaranduba (manilkara huberi) 2.5 metro 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm

In [146]:
print("ESPECIE:", [ent.text for ent in doc.spans["especie_annotator"]])
print("ESPECIE:", [ent.text for ent in doc.spans["cientifico_especie_annotator"]])
print("CORTE:", [ent.text for ent in doc.spans["corte_annotator"]])
print("DIMENSAO:", [ent.text for ent in doc.spans["dimensao_annotator"]])

ESPECIE: ['massaranduba']
ESPECIE: ['manilkara huberi']
CORTE: ['vigas', 'tabuas']
DIMENSAO: ['2,5x30x200cm', '10x15cm', '2,5x30x200', '2.5 metro']


In [147]:
spacy_docs_train = list(combined.pipe(spacy_docs_train))

In [148]:
skweak.utils.docbin_writer(
    spacy_docs_train, "artefatos/spacy_docs_train_annotated.bin"
)

Write to artefatos/spacy_docs_train_annotated.bin...done


In [149]:
import skweak

# Now we can load them back, so we don't need to run the labelling functions again
spacy_docs_train = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_train_annotated.bin", spacy_model_name="pt_core_news_lg"
)
spacy_docs_train = list(spacy_docs_train)

In [150]:
majority_voter = skweak.aggregation.MajorityVoter(
    name="doclevel_voter",
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights={"doc_majority": 0.0}  # Exclude document-level annotator
)

# First apply base annotators and majority voting
spacy_docs_train = list(majority_voter.pipe(spacy_docs_train))

# 2. Create Document Majority Annotator
doc_majority = skweak.doclevel.DocumentMajorityAnnotator("doc_majority","doclevel_voter")

# Apply document-level consistency
spacy_docs_train = list(doc_majority.pipe(spacy_docs_train))

In [151]:
doc

madeira serrada massaranduba (manilkara huberi) 2.5 metro 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm

In [152]:
initial_weights = {
    "especie_annotator": .9,
    "cientifico_especie_annotator": 1,
    "corte_annotator": 1,
    "dimensao_annotator": 1,
    "doclevel_voter": 0.5,
    "doc_majority": 0.6
}

hmm = skweak.generative.HMM(
    "hmm", 
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights=initial_weights
)


hmm.fit(spacy_docs_train)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19902 documents
Starting iteration 2


         1 -131838.81912378             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19902 documents
Starting iteration 3


         2 -126730.80043565   +5108.01868813


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19902 documents
Starting iteration 4


         3 -125252.04498185   +1478.75545379


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19902 documents


         4 -124545.51782849    +706.52715336


In [153]:
hmm.pretty_print()

HMM model with following parameters:
Output labels: ['O', 'B-ESPECIE', 'I-ESPECIE', 'B-CORTE', 'I-CORTE', 'B-DIMENSAO', 'I-DIMENSAO']
--------
Start distribution:
O             0.49
B-ESPECIE     0.16
I-ESPECIE     0.00
B-CORTE       0.35
I-CORTE       0.00
B-DIMENSAO    0.00
I-DIMENSAO    0.00
dtype: float64
--------
Transition model:
               O  B-ESPECIE  I-ESPECIE  B-CORTE  I-CORTE  B-DIMENSAO  I-DIMENSAO
O           0.78       0.03       0.00     0.14      0.0        0.05        0.00
B-ESPECIE   0.64       0.00       0.17     0.00      0.0        0.19        0.00
I-ESPECIE   0.86       0.00       0.00     0.00      0.0        0.13        0.00
B-CORTE     0.68       0.20       0.00     0.01      0.0        0.12        0.00
I-CORTE     0.20       0.20       0.00     0.20      0.2        0.20        0.00
B-DIMENSAO  0.02       0.00       0.00     0.00      0.0        0.01        0.97
I-DIMENSAO  0.41       0.03       0.00     0.00      0.0        0.17        0.39
--------
Label

In [154]:
maj_voter = skweak.voting.SequentialMajorityVoter(
    "maj_voter",
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights=initial_weights
)


In [155]:
spacy_docs_train = list(hmm.pipe(spacy_docs_train))

In [156]:
spacy_docs_train = list(maj_voter.pipe(spacy_docs_train))

In [157]:
from spacy import displacy

def render_skweak_entities(doc, annotator_name: str, colors: dict = None):
    """Render entities from skweak annotations with colored highlighting"""
    
    # Get entities from specified annotator
    entities = doc.spans.get(annotator_name, [])
    
    # Default color scheme for your entities
    default_colors = {
        "ESPECIE": "#7FFFD4",  # Aquamarine
        "CORTE": "#FFD700",    # Gold
        "DIMENSAO": "#FFB6C1"  # Pink
    }
    
    # Ensure colors is never None
    colors = colors or default_colors
    
    # Prepare entities for visualization
    displacy_data = {
        "text": doc.text,
        "ents": [
            {
                "start": ent.start_char,
                "end": ent.end_char,
                "label": ent.label_
            } for ent in entities
        ],
        "title": None
    }

    # Render with specified colors (ensure empty dict if no colors)
    displacy.render(
        [displacy_data],
        style="ent",
        manual=True,
        jupyter=True,
        options={"colors": colors or {}}  # Key fix here
    )

# Usage
doc = spacy_docs_train[0]

print("=== HMM Annotations ===")
render_skweak_entities(doc, "hmm")


print("\n=== Majority Voter Annotations ===")
render_skweak_entities(doc, "maj_voter")

=== HMM Annotations ===



=== Majority Voter Annotations ===


In [158]:
from skweak.utils import docbin_writer
docbin_writer(spacy_docs_train, "artefatos/spacy_docs_train_annotated.bin")

Write to artefatos/spacy_docs_train_annotated.bin...done


## Fazer analise dos que não tiveram nenhum tipo de anotação, também de conflitos para entender possíveis melhoras.

## Make a new model to using weak labels

Ser capaz de generalizar, além do regex

In [159]:
from skweak.utils import docbin_reader

# Load your unlabeled documents with weak annotations
spacy_docs_train = list(docbin_reader("artefatos/spacy_docs_train_annotated.bin", 
                                     spacy_model_name="pt_core_news_lg"))

# Verify annotations
doc = spacy_docs_train[0]
print("Entities:", [(ent.text, ent.label_) for ent in doc.spans["hmm"]])

Entities: [('angelim verm', 'ESPECIE'), ('ripas', 'CORTE')]


In [160]:
def convert_to_iob(docs, annotator_name="hmm"):
    """Convert skweak annotations to IOB format"""
    iob_data = []
    for doc in docs:
        tokens = [token.text for token in doc]
        labels = ["O"] * len(tokens)
        
        # Process each entity span
        for span in doc.spans[annotator_name]:
            label = span.label_
            start = span.start
            end = span.end
            
            labels[start] = f"B-{label}"
            for i in range(start+1, end):
                labels[i] = f"I-{label}"
                
        iob_data.append({"tokens": tokens, "labels": labels, "text": doc.text})
    
    return iob_data

# Convert all training docs
train_data = convert_to_iob(spacy_docs_train, annotator_name="hmm")

In [161]:
import ast
def convert_argilla_to_iob(argilla_data):
    """Convert Argilla format to IOB format"""
    iob_data = []
    
    for example in argilla_data:
        text = example["text"]
        entities = example.get("span_label.responses", [])
        
        if isinstance(entities, str):
            try:
                clean_str = entities.replace("array(", "").replace(", dtype=object)", "")
                entities = ast.literal_eval(clean_str)
            except:
                entities = []

        doc = nlp(text)
        tokens = [token.text for token in doc]
        labels = ["O"] * len(tokens)

        for entity in entities:
            if not isinstance(entity, dict):
                continue

            start = entity["start"]
            end = entity["end"]
            label = entity["label"]

            start_token = None
            for token in doc:
                if token.idx >= start:
                    break
                start_token = token.i
            if start_token is None:
                start_token = 0

            end_token = None
            for token in doc:
                if token.idx + len(token) > end:
                    break
                end_token = token.i
            if end_token is None:
                end_token = len(doc) - 1

            if start_token <= end_token:
                labels[start_token] = f"B-{label}"
                for i in range(start_token + 1, end_token + 1):
                    labels[i] = f"I-{label}"

        iob_data.append({
            "tokens": tokens,
            "labels": labels,
            "text": text
        })
    
    return iob_data

val_data = convert_argilla_to_iob(df_val.to_dict("records"))
test_data = convert_argilla_to_iob(df_test.to_dict("records"))

In [162]:
from datasets import Dataset, Features, Sequence, ClassLabel, Value

labels = ["O", "B-ESPECIE", "I-ESPECIE", "B-CORTE", "I-CORTE", "B-DIMENSAO", "I-DIMENSAO"]

# Create dataset
dataset_train = Dataset.from_list(train_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))
dataset_val = Dataset.from_list(val_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))
dataset_test = Dataset.from_list(test_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))



Casting the dataset: 100%|██████████| 282/282 [00:00<00:00, 56393.33 examples/s]


In [163]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        max_length=512
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset_train = dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_val = dataset_val.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, max_length=512, return_tensors="pt")


Map: 100%|██████████| 282/282 [00:00<00:00, 10888.08 examples/s]


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=len(labels),
    id2label={i: label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)}
)

training_args = TrainingArguments(
    output_dir="./artefatos/bert-base-ner-true-labels",  # Directory to save model checkpoints and logs
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=32,  # Batch size for evaluation
    weight_decay=0.01,  # Weight decay for regularization
    seed=271828,  # Random seed for reproducibility
    bf16=True,  # Use bfloat16 precision for training (if supported by hardware)
    fp16=False,  # Use mixed precision training with FP16 (if supported by hardware)
    save_total_limit=1,  # Limit the total number of saved checkpoints
    logging_steps=1,  # Log training metrics every step
    eval_steps=1,  # Evaluate the model every step
    save_steps=1,  # Save the model every step
    metric_for_best_model="eval_loss",  # Metric to determine the best model
    greater_is_better=False,  # Higher metric value is better
    logging_strategy="steps",  # Log metrics at each step
    eval_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    do_train=True,  # Perform training
    do_eval=True,  # Perform evaluation
    gradient_accumulation_steps=1,  # Accumulate gradients over multiple steps
    push_to_hub=False,  # Do not push the model to the Hugging Face Hub
    learning_rate=2e-6,  # Learning rate for the optimizer
    overwrite_output_dir=True,  # Overwrite the output directory if it exists
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Tales\Python\Madeira-Fraca\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.388400,1.651983
2,0.099900,1.946672
3,0.191900,2.135130
4,0.103400,2.207854
5,0.076300,2.236130


c:\Users\Tales\Python\Madeira-Fraca\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Tales\Python\Madeira-Fraca\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Tales\Python\Madeira-Fraca\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Tales\Python\Madeira-Fraca\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation s

TrainOutput(global_step=3285, training_loss=0.19342311534355855, metrics={'train_runtime': 761.1559, 'train_samples_per_second': 138.014, 'train_steps_per_second': 4.316, 'total_flos': 1688945476785864.0, 'train_loss': 0.19342311534355855, 'epoch': 5.0})

In [165]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner", 
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

text = "madeira serrada massaranduba (manilkara huberi) 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm"
results = ner_pipeline(text)

print("Entities detected:")
for entity in results:
    print(f"{entity['word']} -> {entity['entity_group']}")

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entities detected:
vig -> CORTE
10x -> DIMENSAO
##15cm -> DIMENSAO
tabu -> CORTE
2, -> DIMENSAO
5x -> DIMENSAO
##30x -> DIMENSAO
##200c -> DIMENSAO


problemas tokenizer não é o mesmo, faz sentido quebrar tanto nesse, pensar em workarounds, ou em formas de treinar que sirvam, pra isso.<br>
voltar a olhar o que o professor para me inspirar, pq tambem foi com base em palavra.